In [35]:
import os
from trackml.dataset import load_event
from trackml.score import score_event
from IPython.display import display
import tensorflow as tf
import pandas as pd
import numpy as np
import glob
from sklearn import preprocessing
from sklearn import cluster
import hdbscan

In [71]:
train = np.unique([p.split('-')[0] for p in sorted(glob.glob(os.path.join('..', 'input', 'train_1' , '**')))])
test = np.unique([p.split('-')[0] for p in sorted(glob.glob(os.path.join('..', 'input', 'test' , '**')))])
sub = pd.read_csv(os.path.join('..', 'input', 'sample_submission.csv'))

In [59]:
hits, cells, particles, truth = load_event(train[10])

In [60]:
hits = hits.merge(cells, how='left', on='hit_id')

In [61]:
hits = hits.merge(truth[['hit_id', 'particle_id']], how='left', on='hit_id')

In [62]:
hits

,hit_id,x,y,z,volume_id,layer_id,module_id,ch0,ch1,value,particle_id
0,1,-95.104897,-11.221400,-1502.5,7,2,1,242,1167,0.195064,211675367239319552
1,1,-95.104897,-11.221400,-1502.5,7,2,1,242,1168,0.109527,211675367239319552
2,2,-54.898102,-9.611880,-1502.5,7,2,1,273,453,0.172011,689056171826348032
3,2,-54.898102,-9.611880,-1502.5,7,2,1,273,452,0.152273,689056171826348032
4,3,-58.008400,3.083220,-1502.5,7,2,1,15,489,0.018446,729583345792450560
5,3,-58.008400,3.083220,-1502.5,7,2,1,15,490,0.328385,729583345792450560
6,4,-97.298401,-8.184180,-1502.5,7,2,1,178,1202,0.258261,698059110473531392
7,5,-64.370499,-11.561200,-1502.5,7,2,1,297,623,0.099925,0
8,5,-64.370499,-11.561200,-1502.5,7,2,1,297,622,0.113521,0
9,5,-64.370499,-11.561200,-1502.5,7,2,1,297,624,0.121676,0


In [40]:
hits['particle_id'].value_counts().head()

0                     93327
4519473826496512        198
508914729352167424      193
454866860899303424      192
4504286822137856        186
Name: particle_id, dtype: int64

In [57]:
hits['hit_id'].nunique()

138568

In [41]:
(hits['particle_id'] == 0 ).sum()/len(hits)

0.161716085835482

In [42]:
particle_stats = hits.groupby('particle_id')['value'].agg(['mean', 'var'])

In [43]:
particle_stats['var'] / particle_stats['mean']

particle_id
0                     0.372465
4503805785800704      0.509687
4503943224754176      0.565381
4504011944230912      0.576688
4504080663707648      0.573085
4504149383184384      0.609714
4504218102661120      0.006492
4504218119442433      0.000000
4504218119446530      0.000000
4504286822137856      0.089832
4504286838919169      0.000000
4504286838923266      0.000000
4504355541614592      0.331725
4504424261091328      0.005151
4504630419521536      0.017355
4504630436302849      0.212703
4504630436306946      0.000000
4504699138998272      0.033365
4504699155783682      0.450986
4504974016905216      0.438697
4505248894812160      0.591622
4505317614288896      0.573459
4505455053242368      0.588388
4505523772719104      0.443508
4505592492195840      0.468306
4505661211672576      0.540792
4505661228453889      0.000000
4505661228457986      0.000000
4505729931149312      0.004772
4505729947930625      0.240297
                        ...   
941253765229445120    0.481

In [44]:
for e in train:
    hits, cells, particles, truth = load_event(e)
    print(len(hits), len(cells), len(particles), len(truth))
    display(hits.head(2))
    display(cells.head(2))
    display(particles.head(2))
    display(truth.head(2))
    break

120939 664996 12263 120939


,hit_id,x,y,z,volume_id,layer_id,module_id
0,1,-64.409897,-7.163700,-1502.5,7,2,1
1,2,-55.336102,0.635342,-1502.5,7,2,1


,hit_id,ch0,ch1,value
0,1,209,617,0.013832
1,1,210,617,0.079887


,particle_id,vx,vy,vz,px,py,pz,q,nhits
0,4503668346847232,-0.009288,0.009861,-0.077879,-0.055269,0.323272,-0.203492,-1,8
1,4503737066323968,-0.009288,0.009861,-0.077879,-0.948125,0.470892,2.010060,1,11


,hit_id,particle_id,tx,ty,tz,tpx,tpy,tpz,weight
0,1,0,-64.411598,-7.164120,-1502.5,250710.000000,-149908.00000,-956385.0000,0.00000
1,2,22525763437723648,-55.338501,0.630805,-1502.5,-0.570605,0.02839,-15.4922,0.00001


In [45]:
for e in test:
    hits, cells = load_event(e, parts=['hits', 'cells'])
    print(len(hits), len(cells))
    display(hits.head(2))
    display(cells.head(2))
    break

119016 657072


,hit_id,x,y,z,volume_id,layer_id,module_id
0,1,-60.826698,-4.16023,-1502.5,7,2,1
1,2,-84.729401,-7.29528,-1502.5,7,2,1


,hit_id,ch0,ch1,value
0,1,155,550,0.268630
1,2,180,978,0.316736


In [78]:
scl = preprocessing.StandardScaler()
# dbscan = cluster.DBSCAN(eps=0.00715, min_samples=1, algorithm='auto', n_jobs=-1)
# dbscan = hdbscan.HDBSCAN(min_cluster_size=4, min_samples=2)
dbscan = hdbscan.HDBSCAN(min_cluster_size=3)
sub = []
for e in test:
#     hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
#     print(truth['particle_id'].nunique())
    hits, cells = load_event(e, parts=['hits', 'cells'])

#     cells.sort_values(by='value', inplace=True, ascending=False)
#     cells.drop_duplicates('hit_id', keep='first', inplace=True)
    hits['event_id'] = int(e[-9:])
#     cells = cells.groupby(by=['hit_id'])['ch0', 'ch1', 'value'].agg(['mean']).reset_index()
#     cells = cells[['hit_id','value']]
#     hits = hits.merge(cells, how='left', on='hit_id')

    #https://www.kaggle.com/mikhailhushchyn/dbscan-benchmark
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values
    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r
    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r
#     data = scl.fit_transform(hits[['x2', 'y2', 'z2']].values)
    data = scl.fit_transform(hits[['x2', 'y2', 'z2']].values)
#     data = np.hstack((scl.fit_transform(hits[['x2', 'y2', 'z2']].values), hits[['value']].values))
#     data = hits[['value']].values
    hits['track_id'] = dbscan.fit_predict(data)
    
    hits['track_id'] = hits['track_id'] + 1
    print(score_event(truth, hits))
    sub.append(hits[['event_id','hit_id','track_id']].copy())
    print(e, len(hits['track_id'].unique()))
    #break
    
sub = pd.concat(sub, ignore_index=True)

sub[['event_id','hit_id','track_id']].to_csv('submission-001.csv', index=False)

0.0
../input/test/event000000000 10369
0.0
../input/test/event000000001 10700
0.0
../input/test/event000000002 9214
0.0
../input/test/event000000003 9534
0.0
../input/test/event000000004 10439
0.0
../input/test/event000000005 9413
0.0
../input/test/event000000006 9374
0.0
../input/test/event000000007 10205
0.0
../input/test/event000000008 10003
0.0
../input/test/event000000009 9929
0.0
../input/test/event000000010 10152
0.0
../input/test/event000000011 9319
0.0
../input/test/event000000012 9110
0.0
../input/test/event000000013 10931
0.0
../input/test/event000000014 9047
0.0
../input/test/event000000015 7821
0.0
../input/test/event000000016 8461
0.0
../input/test/event000000017 11572
0.0
../input/test/event000000018 8573
0.0
../input/test/event000000019 9098
0.0
../input/test/event000000020 6993
0.0
../input/test/event000000021 9294
0.0
../input/test/event000000022 10540
0.0
../input/test/event000000023 10170
0.0
../input/test/event000000024 10207
0.0
../input/test/event000000025 9583
0